# Monowai CC tests

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from obspy import read_inventory
import os
import ccf

In [ ]:
%matplotlib notebook

### Open a file

In [ ]:
def filename(path:str, r1:dict, r2:dict, time:pd.datetime):
    pair = '-'.join([ccf.helpers.merge_receiver(r1), ccf.helpers.merge_receiver(r2)])
    return os.path.join(path,pair,'{p}.{y:04d}.{d:03d}.nc'.format(p=pair,y=time.year,d=time.dayofyear))

In [ ]:
time = pd.to_datetime('2015-01-15')

hydro = {
    'network'  : 'IM',
    'station'  : 'H10N1',
    'location' : '',
    'channel'  : 'EDH',
}
seism = {
    'network'  : 'AU',
    'station'  : 'NIUE',
    'location' : '',
    'channel'  : 'BHZ',
}

vel = dict( min=1.46, max=1.50 )

bhz = ccf.open_dataset( filename(pwd, hydro, seism, time) )
bhr = None

assert bhz, 'No data found!'

if bhz:
    ccf.bias_correct_dataset(bhz,unbiased_var='cc_w')
if bhr:
    ccf.bias_correct_dataset(bhr,unbiased_var='cc_w')

### CCF noise window

In [ ]:
plt.figure(figsize=[9,4])
bhz.cc_w.loc[{'time':bhz.time[1]}].plot.line(x='lag',add_legend=False,color='orange')
bhz.cc.loc[{'time':bhz.time[1]}].plot.line(x='lag',add_legend=False,color='blue')

In [ ]:
noise_lim = pd.to_timedelta([bhz.time.window_length * v for v in [.2, .3]], unit='s')

noise_win = (
    ( bhz.lag/ccf.helpers.one_second >= noise_lim[0]/ccf.helpers.one_second ) & 
    ( bhz.lag/ccf.helpers.one_second <= noise_lim[1]/ccf.helpers.one_second )
)

signal_lim = pd.to_timedelta((bhz.distance.values/vel['max'],bhz.distance.values/vel['min']), unit = 's' )

signal_win = (
    ( bhz.lag/ccf.helpers.one_second >= signal_lim[0]/ccf.helpers.one_second ) & 
    ( bhz.lag/ccf.helpers.one_second <= signal_lim[1]/ccf.helpers.one_second )
)

valid_win = ( abs(bhz.lag/ccf.helpers.one_second) <= noise_lim[1]/ccf.helpers.one_second )

i = 1

print('signal : ', signal_lim)
print('noise  : ', noise_lim)

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc.loc[{'time':bhr.time[i],'lag':bhr.lag[valid_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc.loc[{'time':bhz.time[i],'lag':bhz.lag[valid_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc.loc[{'time':bhr.time[i],'lag':bhr.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc.loc[{'time':bhz.time[i],'lag':bhz.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc_w.loc[{'time':bhr.time[i],'lag':bhr.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc_w.loc[{'time':bhz.time[i],'lag':bhz.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='orange')

In [ ]:
plt.figure(figsize=[9,4])
if bhr:
    bhr.cc_w.loc[{'time':bhr.time[i],'lag':bhr.lag[noise_win]}].plot.line(x='lag',add_legend=False,color='blue')
if bhz:
    bhz.cc_w.loc[{'time':bhz.time[i],'lag':bhz.lag[noise_win]}].plot.line(x='lag',add_legend=False,color='orange')

### SNR

In [ ]:
if bhz:
    bhz['snr'] = ( 
        ccf.postprocess.rms(bhz.cc_w.loc[{'lag':bhz.lag[signal_win]}]) / 
        ccf.postprocess.rms(bhz.cc_w.loc[{'lag':bhz.lag[noise_win]}])
    )
    print(bhz.snr)
    
if bhr:
    bhr['snr'] = ( 
        ccf.postprocess.rms(bhr.cc_w.loc[{'lag':bhr.lag[signal_win]}]) / 
        ccf.postprocess.rms(bhr.cc_w.loc[{'lag':bhr.lag[noise_win]}])
    )
    print(bhr.snr)

### Select CC based on SNR

In [ ]:
snr_min = 1

plt.figure(figsize=[9,4])
if bhr and any(bhr.snr >= snr_min):
    bhr.cc_w.loc[{'time':bhr.time[snr_w >= snr_min],'lag':bhr.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='blue',alpha=0.3)
if bhz and any(bhz.snr >= snr_min):
    bhz.cc_w.loc[{'time':bhz.time[snr_w >= snr_min],'lag':bhz.lag[signal_win]}].plot.line(x='lag',add_legend=False,color='orange',alpha=0.3)